In [ ]:
%pip uninstall nomad -y
%pip install git+https://github.com/Watts-Lab/nomad.git@data-gen

In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr
from datetime import datetime, timedelta
import calendar
from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from functools import partial
import copy

import nomad.city_gen as cg
from nomad.city_gen import City, Building, Street
import nomad.traj_gen
from nomad.traj_gen import Agent, Population
from nomad.constants import DEFAULT_SPEEDS, FAST_SPEEDS, SLOW_SPEEDS, DEFAULT_STILL_PROBS
from nomad.constants import FAST_STILL_PROBS, SLOW_STILL_PROBS, ALLOWED_BUILDINGS

import os
os.environ['TZ'] = 'UTC'

### Generate N agents

In [2]:
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)


def garden_city_to_lat_long(agent, 
                            sparse_traj=True, 
                            full_traj=True):
    if sparse_traj:
        df = agent.sparse_traj
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['latitude'], df['longitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df['date'] = df['local_timestamp'].dt.date  # for partitioning purposes

        df = df[['identifier', 'unix_timestamp', 'date', 'latitude', 'longitude']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.sparse_traj = df

    if full_traj:
        df = agent.trajectory
        df['x'] = 15*df['x'] + 4265699
        df['y'] = 15*df['y'] - 4392976

        df['latitude'], df['longitude'] = transformer.transform(
            df['x'].values, df['y'].values)

        df['date'] = df['local_timestamp'].dt.date  # for partitioning purposes

        df = df[['identifier', 'unix_timestamp', 'date', 'latitude', 'longitude']]
        df = df.rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'})
        df = df.reset_index(drop=True)

        agent.trajectory = df
    return None

In [3]:
%%time

N = 5
npr.seed(100)

city = cg.load('garden-city.pkl')
population_n = Population(city)
population_n.generate_agents(N,
                             start_time=datetime(2024, 1, 1, hour=8, minute=0),
                             dt=1, seed=123)

def generate_agent_trajectory(agent_id, agent, seed):
    
    # May want to put more thought into the ranges
    beta_start = npr.uniform(60, 1200)
    beta_duration = npr.uniform(15, 180)
    beta_ping = npr.uniform(5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    
    for month in range(1,4):
        days = calendar.monthrange(2024, month)[1]
        population_n.generate_trajectory(agent, 
                                         T=datetime(2024, month, days, hour=23, minute=59), 
                                         seed=seed)
    
        agent.sample_traj_hier_nhpp(*param, 
                                    seed=seed,
                                    reset_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

with ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(generate_agent_trajectory, agent_id, agent, i)
        for i, (agent_id, agent) in enumerate(shared_roster.items())
    ]
    for future in futures:
        agent_id, agent = future.result()
        shared_roster[agent_id] = agent

population_n.roster = dict(shared_roster)

CPU times: user 1.5 s, sys: 274 ms, total: 1.78 s
Wall time: 42.3 s


In [4]:
partition_cols = {
    'sparse_traj': ['date'],
    'diaries': ['identifier']
}

population_n.save_pop(bucket='synthetic-raw-data', 
                      prefix='1mil-agents/',
                      save_full_traj=False,
                      save_sparse_traj=True,
                      save_homes=True,
                      save_diaries=True,
                      partition_cols=partition_cols)

In [6]:
# Load the Parquet file

s3_path = "s3://synthetic-raw-data/1mil-agents/sparse_trajectories.parquet/"
df = pd.read_parquet(s3_path) 

In [7]:
df

,uid,timestamp,latitude,longitude,date
0,wizardly_joliot,1704119040,38.321697,-36.667286,2024-01-01
1,wizardly_joliot,1704119520,38.321642,-36.667270,2024-01-01
2,wizardly_joliot,1704119580,38.321649,-36.667295,2024-01-01
3,wizardly_joliot,1704119940,38.321634,-36.667210,2024-01-01
4,wizardly_joliot,1704120540,38.321699,-36.667381,2024-01-01
...,...,...,...,...,...
2260,admiring_curie,1711914840,38.320422,-36.666893,2024-03-31
2261,admiring_curie,1711915440,38.320456,-36.666824,2024-03-31
2262,admiring_curie,1711915860,38.320438,-36.666820,2024-03-31
2263,admiring_curie,1711916760,38.320428,-36.666884,2024-03-31
